In [ ]:
import sys
import os
import itertools

import numpy as np
import matplotlib.pyplot as plt

import scipy.optimize as spopt
import scipy.integrate as spi
import scipy.stats as sps

import tensorflow as tf
import pandas as pd

sys.path.append("../../ndsvae/")
import ndsvae as ndsv

sys.path.append("../")
import util
import dsa

%matplotlib inline

In [ ]:
config = "ns_3_mreg_3_msub_0_nf_32"
conn = "linw"
preproc = "dicer"
modelname = "AB"

ds = ndsv.Dataset.from_file(f"../run/hcp/hcp100_{conn}_{preproc}/dataset.npz")
run = util.select_run_fc(f"hcp100_{conn}_{preproc}", modelname, config, [0,1], "hcp")
direc = f"../run/hcp/hcp100_{conn}_{preproc}/model{modelname}/{config}/run{run:02d}"
params = util.load_params(os.path.join(direc, "parameters"), np.r_[:100])

model = util.get_model(modelname, config, ds)
model.load_weights(f"../run/hcp/hcp100_{conn}_{preproc}/model{modelname}/{config}/run{run:02d}/fit/model")

In [ ]:
nsub, nreg, _, nt = ds.y.shape

In [ ]:
def deduplicate(xs, threshold):
    xds = []    
    for x in xs:  
        if np.any([(np.linalg.norm(x - xd) < threshold) for xd in xds]):
            continue
        else:
            xds.append(x)            
    return np.array(xds)


def find_fixed_points(model, w, thetareg, thetasub, us=0., n=10, init_range=(-2,2), threshold=1e-4):
    nreg = model.nreg
    
    assert thetareg.shape == (nreg, model.mreg)
    assert thetasub.shape == (model.msub,)
    
    x0 = np.random.uniform(init_range[0], init_range[1], size=(n, nreg * model.ns))
    Ap = model.Ap.numpy()
    bp = model.bp.numpy()
    w = w.astype('float32')
    
    has_network = int(model.source_model.network_input)
    has_shared  = int(model.source_model.shared_input)    
    input_size = (model.ns + model.mreg + model.msub + has_network + has_shared)
    
    xaugf = np.zeros((nreg, input_size))
    xaugf[:, model.ns:model.ns+model.mreg] = thetareg
    xaugf[:, model.ns+model.mreg:model.ns+model.mreg+model.msub] = thetasub
    if has_shared: xaugf[:, -1] = us    
    
    def f(x):
        x = x.reshape((nreg, model.ns))
        y = (np.dot(Ap, x.T) + bp)[0,:]
        u = np.dot(w, y)
        
        xaug = np.copy(xaugf)
        xaug[:, :model.ns] = x
        if has_network:
            xaug[:, -has_network-has_shared] = u
        
        fx = model.source_model.f(xaug).numpy().reshape(nreg * model.ns)
        return fx
   
    @tf.function
    def jacobian(x):
        x = tf.cast(x, 'float32')
        with tf.GradientTape(persistent=True, watch_accessed_variables=False) as g:
            g.watch(x)
            xr = tf.reshape(x, (nreg, model.ns))
            y = tf.tensordot(Ap, xr, axes=[[1], [1]])[0] + bp
            u = tf.linalg.matvec(w, y)            

            if not has_network:
                xaug = tf.concat([xr, xaugf[:, model.ns:]], axis=1)
            else:
                xaug = tf.concat([xr,
                                  xaugf[:, model.ns:model.ns+model.mreg+model.msub],
                                  u[:,None],
                                  xaugf[:, model.ns+model.mreg+model.msub+has_network:],
                                 ], axis=1)
        
            fx = model.source_model.f(xaug)
            fx = tf.reshape(fx, (nreg*model.ns,))
                        
        jac = g.jacobian(fx, x)
        return jac                   
        
    xs = []
    fails = 0
    for i in range(n):
        sol = spopt.root(f, x0[i], jac=jacobian, method='hybr', tol=1e-6)

        if not sol.success:
            fails += 1
        else:
            xs.append(sol.x)
        
    xs = np.array(xs)    
    xs = deduplicate(xs, threshold)
    xs = xs.reshape(-1, nreg, model.ns)
    
    # Evaluate stability
    stability = np.zeros(len(xs), dtype=bool)
    eigvals = np.zeros((len(xs), nreg*model.ns), dtype=complex)
    eigvecs = np.zeros((len(xs), nreg*model.ns, nreg*model.ns), dtype=complex)
    
    for i, x in enumerate(xs):
        jac = jacobian(np.reshape(x, nreg*model.ns)).numpy()
        evals, evecs = np.linalg.eig(jac)
        order = np.argsort(-evals.real)
        eigvals[i] = evals[order]
        eigvecs[i] = evecs[:,order]
        stability[i] = np.all(np.real(eigvals[i]) < 0.)
    
    return xs, stability, eigvals, eigvecs

In [ ]:
def is_converging(x, t, nsteps, threshold):
    nsteps = min(len(t), nsteps)
    dist = np.linalg.norm(x[:,:,-nsteps:] - x[:,:,[-1]], axis=1)
    converging = np.zeros(x.shape[0], dtype=bool)
    
    for i in range(x.shape[0]):
        if np.all(dist[i] < threshold):
            converging[i] = True
        else:
            last_above = np.where(dist[i] >= threshold)[0][-1]
            dist_above = dist[i,:last_above+1]
            converging[i] = np.all(dist_above[:-1] > dist_above[1:])
            
    return converging
    
    
def is_periodic(x, t, threshold, min_period=0.):
    n = len(t)
    max_offset = int(len(t) / 4)
    
    diff = np.zeros((max_offset, n-max_offset))
    for i in range(n-max_offset):
        diff[:,i] = np.linalg.norm(x[:,[i]] - x[:,i:i+max_offset], axis=0)
    
    min_ind = np.argwhere(t > t[0] + min_period)[0][0]
    candidate_ind = np.where(np.max(diff[min_ind:,:], axis=1) < threshold)[0][0] + min_ind
    
    if np.max(diff[candidate_ind,:]) < threshold:
        return True, t[candidate_ind] - t[0]
    else:
        return False, np.nan


def find_attractors(model, w, thetareg, thetasub, us=0., n=10, init_range=(-2,2), T1=400, T2=1200):
    
    nreg = model.nreg    
    assert thetareg.shape == (nreg, model.mreg)
    assert thetasub.shape == (model.msub,)
    
    x0 = np.random.uniform(init_range[0], init_range[1], size=(n, nreg * model.ns))
    Ap = model.Ap.numpy()
    bp = model.bp.numpy()
    w = w.astype('float32')
    
    has_network = int(model.source_model.network_input)
    has_shared  = int(model.source_model.shared_input)    
    input_size = (model.ns + model.mreg + model.msub + has_network + has_shared)
    
    xaugf = np.zeros((nreg, input_size))
    xaugf[:, model.ns:model.ns+model.mreg] = thetareg
    xaugf[:, model.ns+model.mreg:model.ns+model.mreg+model.msub] = thetasub
    if has_shared: xaugf[:, -1] = us    
    
    def f(t, x):
        x = x.reshape((nreg, model.ns))
        y = (np.dot(Ap, x.T) + bp)[0,:]
        u = np.dot(w, y)
        
        xaug = np.copy(xaugf)
        xaug[:, :model.ns] = x
        if has_network:
            xaug[:, -has_network-has_shared] = u
        
        fx = model.source_model.f(xaug).numpy().reshape(nreg * model.ns)
        return fx
    
    # Simulate all
    xs = []
    ts = []
    
    converging = np.zeros(n, dtype=bool)    
    periodic = np.zeros(n, dtype=bool)
    period = np.full(n, np.nan)    
    
    for i in range(n):
        sol = spi.solve_ivp(f, (0, T1), x0[i], method='RK45', max_step=1)
        if not sol.success:
            raise ValueError(message)
    
        x = np.reshape(sol.y, (nreg, model.ns, -1))
        t = sol.t
    
        converging[i] = is_converging(np.reshape(x, (1, nreg*model.ns, len(t))), t, int(0.1*len(t)), 1e-3)[0]
        
        if not converging[i]:
            sol2 = spi.solve_ivp(f, (0, T2), x0[i], method='RK45', max_step=0.5)            
            if not sol2.success:
                raise ValueError(message)
    
            x = np.reshape(sol2.y, (nreg, model.ns, -1))
            t = sol2.t
            
            # Recheck convergence using this detailed simulation            
            converging[i] = is_converging(np.reshape(x, (1, nreg*model.ns, len(t))), t, int(0.1*len(t)), 1e-3)[0]
            mask = t > T1
            if not converging[i]:
                periodic[i], period[i] = is_periodic(sol2.y[:,mask], sol2.t[mask], 0.1, min_period=2.)
                
        ts.append(t)
        xs.append(x)
    
    return (ts, xs, converging, periodic, period)

In [ ]:
nsubs = 100
nrand = 1

rows_fp = []
rows_at = []
fixed_points = []
eigenvalues = []
thetareg = np.zeros((nsubs, nrand, nreg, model.mreg))

ifp = 0

for isub in range(nsubs):
    print(f"Subject {isub}: ", end='')
    for irand in range(nrand):
        print(f"{irand}", end=' ', flush=True)
        
        for us in [-1, 0, 1]:
            treg = np.random.normal(params.thetareg[isub,:,:,0], params.thetareg[isub,:,:,1])
            tsub = np.zeros(1)
            thetareg[isub,irand,:,:] = treg

            fps, stability, eigvals, eigvecs = dsa.find_fixed_points_network(model, ds.w[isub], treg, tsub,
                                                                             us=us, n=100, init_range=(-2,2))

            ts, xs, converging, periodic, period = dsa.find_attractors_network(model, ds.w[isub], treg, tsub,
                                                                               us=us, n=100, init_range=(-2,2), 
                                                                               T1=400, T2=1200)

            for i, fp in enumerate(fps):    
                rows_fp.append(dict(isub=isub, irand=irand, us=us, n=i, ifp=ifp, stable=stability[i]))
                fixed_points.append(fps[i])
                eigenvalues.append(eigvals[i])        
                ifp += 1

            rows_at.append(dict(isub=isub, irand=irand, us=us,
                                has_converging=np.any(converging),
                                has_nonconverging=np.any(~converging),
                                has_periodic=np.any(periodic[~converging]),
                                has_aperiodic=np.any(~periodic[~converging]),
                               ))
    print("")

df_fp = pd.DataFrame(rows_fp)
df_at = pd.DataFrame(rows_at)
fixed_points = np.array(fixed_points)
eigenvalues = np.array(eigenvalues)    

df_fp.to_csv("./res/subj_fp2.csv")
df_at.to_csv("./res/subj_at2.csv")
np.savez("./res/subj_fp2.npz", fixed_points=fixed_points, eigenvalues=eigenvalues, thetareg=thetareg)

In [ ]:
## Add u

import itertools

df_fp['us'] = 0.

uss = itertools.cycle([-1, 0, 1])
us = None

lastn = 0
for i in range(len(df_fp)):
    if df_fp.loc[i, 'n'] <= lastn: 
        us = next(uss)
    df_fp.loc[i, 'us'] = us    
    lastn = df_fp.loc[i, 'n']

In [ ]:
df_fp = pd.read_csv("./res/subj_fp.csv")
df_at = pd.read_csv("./res/subj_at.csv")
data = np.load("./res/subj_fp.npz")
fixed_points = data['fixed_points']
eigenvalues = data['eigenvalues']
thetareg = data['thetareg']

df_fp['us'] = 0.

## Multistable configurations ?

In [ ]:
for isub in range(nsub):
    for irand in range(4):
        for us in [-1, 0, 1]:
            dff = df_fp[(df_fp.isub == isub) & (df_fp.irand == irand) & (df_fp.us == us)]
            if np.any(dff.n >= 1) or np.any(~dff.stable):            
                print(f"Subject {isub}, init {irand}, us = {us}")

                for i, (_, row) in enumerate(dff.iterrows()):
                    print(f"    FP{i}: {'Stable' if row.stable else 'Unstable'}")

                ifps = dff.ifp.to_list()
                for i, ifp1 in enumerate(ifps):
                    for j, ifp2 in enumerate(ifps[:i]):
                        x1 = fixed_points[ifp1]
                        x2 = fixed_points[ifp2]
                        l2 = np.linalg.norm(x1-x2)
                        linf = np.linalg.norm(x1-x2, ord=np.inf)
                        print(f"    Distance FP{j}-FP{i}: L2 {l2:8.5f}   Linf {linf:8.5f}")

                print("")

In [ ]:
df_fp[df_fp.isub == 91]

## Attractors?

In [ ]:
df_at

In [ ]:
print(f"Converging:    {sum(df_at.has_converging)} / {len(df_at)}")
print(f"Nonconverging: {sum(df_at.has_nonconverging)} / {len(df_at)}")
print(f"Periodic:      {sum(df_at.has_periodic)} / {len(df_at)}")
print(f"Aperiodic:     {sum(df_at.has_aperiodic)} / {len(df_at)}")